<a href="https://colab.research.google.com/github/rmiller34/ATMS-597-Project-5/blob/master/Tree_Project_5_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')
#change directory to the right spot
import os 
os.chdir('/content/gdrive/My Drive/ATMS_597_Project_5')

### Read all years data file
from datetime import datetime
import pandas as pd

!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob
import numpy as np


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import brier_score_loss
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np


In [63]:

# Read data
filename= 'decoded_metar_FINAL.csv'
decoded_metar_dataset=pd.read_csv(filename,dtype={'current_wx1':str})
decoded_metar_dataset

##Rain/Liquid: RA, FZRA, TSRA =value 0  Snow/Freezing: SN, SNDZ=value 1)



,Unnamed: 0,date_time,station_id,latitude,longitude,elevation,date_time.1,wind_direction,wind_speed,current_wx1,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,eastward_wind,northward_wind,month,fzn_or_liq
0,1,2000-01-01 00:55:00,KMSN,43.13,-89.35,261,2000-01-01 00:55:00,320.0,6.0,-SN,-6.0,-9.0,29.92,71,0,0,3.856726,-4.596267,1,1
1,2,2000-01-01 01:00:00,KMSN,43.13,-89.35,261,2000-01-01 01:00:00,340.0,7.0,-SN,-5.0,-9.0,29.92,71,0,0,2.394141,-6.577848,1,1
2,3,2000-01-01 01:05:00,KMSN,43.13,-89.35,261,2000-01-01 01:05:00,330.0,10.0,-SN,-5.0,-10.0,29.92,71,0,0,5.000000,-8.660254,1,1
3,4,2000-01-01 01:10:00,KMSN,43.13,-89.35,261,2000-01-01 01:10:00,320.0,8.0,-SN,-6.0,-10.0,29.93,71,0,0,5.142301,-6.128356,1,1
4,5,2000-01-01 01:15:00,KMSN,43.13,-89.35,261,2000-01-01 01:15:00,320.0,8.0,-SN,-6.0,-10.0,29.93,71,0,0,5.142301,-6.128356,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175199,412969,2019-12-31 15:25:00,KMSN,43.13,-89.35,261,2019-12-31 15:25:00,300.0,8.0,-SN,-4.0,-8.0,29.69,71,0,0,6.928203,-4.000000,12,1
175200,412970,2019-12-31 15:30:00,KMSN,43.13,-89.35,261,2019-12-31 15:30:00,300.0,9.0,-SN,-4.0,-8.0,29.69,71,0,0,7.794229,-4.500000,12,1
175201,412971,2019-12-31 15:35:00,KMSN,43.13,-89.35,261,2019-12-31 15:35:00,300.0,8.0,-SN,-4.0,-8.0,29.69,71,0,0,6.928203,-4.000000,12,1
175202,412972,2019-12-31 15:40:00,KMSN,43.13,-89.35,261,2019-12-31 15:40:00,290.0,10.0,-SN,-4.0,-8.0,29.69,71,0,0,9.396926,-3.420201,12,1


In [64]:
where_snow_rain = [i for i,x in enumerate(decoded_metar_dataset['current_wx1'].to_list()) if ('RA' in x) | ('SN' in x)]
rain_snow = decoded_metar_dataset.iloc[where_snow_rain]
rain_snow.index = pd.to_datetime(rain_snow.index)
exclude_both = [i for i,x in enumerate(rasn['current_wx1'].to_list()) if 'SNRA' not in x]
rain_snow = rain_snow.iloc[exclude_both]
set(rain_snow['current_wx1'].to_list())

{'+RA',
 '+SN',
 '+TSRA',
 '-FZRA',
 '-RA',
 '-SN',
 '-TSRA',
 'FZRA',
 'RA',
 'SN',
 'TSRA'}

In [65]:
rain_snow['snow'] = 0
where_snow = [i for i,x in enumerate(rain_snow['current_wx1'].to_list()) if 'SN' in x]
rain_snow['snow'][where_snow] = 1

rain_snow['day_of_year'] = rain_snow.index.dayofyear

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
features = ['air_temperature', 'wind_speed', 'wind_direction',  
            'dew_point_temperature', 
            'altimeter', 'eastward_wind', 
            'northward_wind', 'day_of_year',
            'snow']

rain_snow_sub = rain_snow.dropna(subset = features)
train_X, valid_X, train_y, valid_y = train_test_split(rain_snow_sub[features[:-1]], rain_snow_sub['snow'], 
                                                  test_size = 0.3, random_state = 50)


In [0]:
dtree    = tree.DecisionTreeClassifier()
model  = dtree.fit(train_X, train_y)
prediction = model.predict(valid_X)

valid_prob = model.predict_proba(valid_X)
train_prob = model.predict_proba(train_X)

In [0]:
def get_climo(y, shape):
    climo = (np.size((y == 1).values.nonzero()))/float(np.size(y))
    prob  = np.zeros((shape))
    prob[:,0] = 1-climo
    prob[:,1] = climo

    return prob

In [0]:

train_climo_prob = get_climo(train_y, train_prob.shape)
valid_climo_prob = get_climo(valid_y, valid_prob.shape)
bsscore_train = 1 - (brier_score_loss(train_y, train_prob[:,1])/brier_score_loss(train_y, train_climo_prob[:,1]))
bsscore_valid = 1 - (brier_score_loss(valid_y, valid_prob[:,1])/brier_score_loss(valid_y, valid_climo_prob[:,1]))

In [71]:
print('Training Brier skill score: ' + str(bsscore_train))
print('Validation Brier skill score: ' + str(bsscore_valid))

Training Brier skill score: 0.9969833900949442
Validation Brier skill score: 0.9546344770308283


In [0]:
#plotting
#fpr, tpr, thresholds = roc_curve(valid_y, valid_prob[:,1])

In [0]:
# plt.plot([0, 1], [0, 1], linestyle='--')
# plt.plot(fpr, tpr)
# plt.show()